Task 3: Training Considerations
Discuss the implications and advantages of each scenario and explain your rationale as to how
the model should be trained given the following:
1. If the entire network should be frozen.
2. If only the transformer backbone should be frozen.
3. If only one of the task-specific heads (either for Task A or Task B) should be frozen.


1.Entire Network Frozen

Implications:

No adaptation of BERT or heads. NO weights are getting updated.
Heads remain untrained (if randomly initialized) → poor task performance unless you’ve already fine‑tuned elsewhere.
Zero gradient computation through the model → fastest possible inference and minimal memory and computation.

Advantages:

lightweight inference: ideal for on‑device or low‑resource deployment.
Safe feature extraction: Can use BERT purely as a static embedding generator and train external classifiers separately.
Since nothing is getting updated, no pretrained knowledge will be lost.

Training: 

None. Can only do inference or use BERT as a static feature extractor.

Rationale:

No risk of overfitting or catastrophic forgetting.
Zero GPU cost for gradient computation.
Use case: Model and heads are already fine‑tuned heads. Aiming for instant deployment, or using the model to do feature‐extraction for a downstream non‑neural classifier.


2.Backbone Frozen, Heads Trainable

Implications:

Fixed sentence & token representations from BERT → heads learn to map those static features to class labels.
Only heads’ parameters update →  fewer trainable weights.
Faster convergence and lower GPU/CPU/memory requirements than full fine‑tuning.

Advantages:

Parameter efficiency: Only train parameters in each head, which is tiny compared to fintuning the entire BERT + heads.
Stability on small data: with limited labeled examples, you reduce overfitting risk.
Rapid prototyping: you can iterate quickly on head architectures or label schemes without touching BERT.

Training:

Freeze all BERT parameters.
Train only the two heads (sent_classifier and token_classifier).

Rationale:

Parameter efficiency: only heads parameters updated.
Stability: low risk of overfitting on small dataset.
Speed: very fast convergence, minimal GPU memory.

Use case:

Rapid prototyping of new label sets.
On‑device fine‑tuning with tight compute/memory budgets.
When your domain is close to BERT’s pretraining.

3.One Head Frozen, Backbone & Other Head Trainable

Implications:

Adapt BERT to better serve the unfrozen task, while preserving the frozen head’s original mapping.
As BERT gtting updated, the frozen head’s inputs shift, which can degrade its performance.
Intermediate compute cost: you still backprop through most of BERT but skip one head.

Advantages:

Task isolation: if one task has abundant data and you want to improve it without touching the other, you can freeze the weaker‑data head.
Transfer learning flexibility: you can “stack” tasks sequentially—e.g., first fine‑tune for NER, then freeze NER head and fine‑tune for classification.
Balanced resource use: you don’t pay the full cost of two‑head training, yet still adapt the shared encoder.

Training:

Freeze BERT plus one task head(NER head).
Train the other head (sentence classification) and optionally the top few BERT layers.

Rationale:

Sequential task addition: you’ve already fine‑tuned NER on a large dataset and want to add a small sentence‐classification dataset without disturbing NER.
Controlled adaptation: you focus model capacity on the new task while preserving an existing head’s behavior.
As BERT getting updated, the frozen head’s inputs drift so extra fine‑tune of that head might be requried later.

Use case:

Curriculum or continual learning pipelines.
When tasks differ in data volume and you want to prioritize one.

Consider a scenario where transfer learning can be beneficial. Explain how you would approach
the transfer learning process, including:
1. The choice of a pre-trained model.
2. The layers you would freeze/unfreeze.
3. The rationale behind these choices.

 Transfer learning gives you the ability leverage a large pretrained Transformer(BERT) so you don’t have to learn everything from scratch. Especially when you only have a small amount of labeled data.

 1.The choice of a pre-trained model.
 
 If the task if not domain specific, we can start with a strong, widely used base model such as bert-base-uncased, roberta-base, or deberta-v3-base. However, if the task is domain specific, let's say clinical data, search for a public variant that’s already been further pre‑trained on that domain.

 2.The layers you would freeze/unfreeze.

 If there is not time and resource contraints, of course we can fintune the entire model. However, that is rarely the case, so the smartest option to start with is to freeze the backbone(most amount of params), and train the heads, with minimal compute. If the fintuning result is not ideal, or the pretrained backbone is not aligning well with the heads, consider unfreezing the top layers, so the model learn more task specific deatails without updating or forgetting its core knowedge. The last option is to unfreeze and retrain the entire model, but that requires a huge dataset and a lot of computing resource.

 3.Rationale Behind These Choices

Catastrophic Forgetting Mitigation:
Starting with frozen weights preserves the pretrained knowledge. Gradual unfreezing prevents large destructive updates.

Data Efficiency:
Small datasets can’t support training ~100 M parameters. By training heads first, you only update a few thousand weights.

Compute & Time Savings:
Frozen layers skip gradient computations, reducing GPU memory and speeding up iterations in early phases.

Domain Adaptation:
If you can, pick a domain‑adapted checkpoint so your “Phase 1” features are already well‑aligned—heads learn faster and generalize better.

Layer‑wise Specialization:
Lower Transformer layers capture universal language features; upper layers capture more task‑specific semantics. Freezing the bottom layers retains generality, unfreezing the top layers focuses adaptation where it’s most effective.

